In [2]:
import pandas as pd
import datetime

In [ ]:
df_record = pd.read_csv("s3://applehealthdashboard/etl/extract/exportRecord.csv")
df_workout = pd.read_csv("s3://applehealthdashboard/etl/extract/exportWorkout.csv")
df_summary = pd.read_csv("s3://applehealthdashboard/etl/extract/exportActivitySummary.csv")

In [12]:
fact_table = pd.read_csv("/Users/ericcollins/healthdashboard_data/etl/transform/fact_table.csv.gz", compression="gzip")
summary_table = pd.read_csv("/Users/ericcollins/healthdashboard_data/etl/transform/summary_table.csv.gz", compression="gzip")

/var/folders/23/bwtd0x3j2qjf60tdmj7g56400000gn/T/ipykernel_1044/1809403198.py:1: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  fact_table = pd.read_csv("/Users/ericcollins/healthdashboard_data/etl/transform/fact_table.csv.gz", compression="gzip")


In [13]:
fact_table

,username,email,activity_category,activity_type_name,unit_name,source_name,creation_ts,start_ts,end_ts,value,device_name,device_manufacturer,device_model,device_hardware,device_software
0,eric,test@test.com,quantity,Height,ft,iPhone (7),2022-07-04 08:41:18 -0700,2022-07-04 08:41:18 -0700,2022-07-04 08:41:18 -0700,5.916670,NaN,NaN,NaN,NaN,NaN
1,eric,test@test.com,workout,StairClimbing,min,Eric’s Apple Watch,2022-07-04 13:36:16 -0700,2022-07-04 13:09:34 -0700,2022-07-04 13:36:16 -0700,26.275945,NaN,NaN,NaN,NaN,NaN
2,eric,test@test.com,quantity,BodyMass,lb,iPhone (7),2022-07-04 08:41:18 -0700,2022-07-04 08:41:18 -0700,2022-07-04 08:41:18 -0700,150.000000,NaN,NaN,NaN,NaN,NaN
3,eric,test@test.com,workout,TraditionalStrengthTraining,min,Eric’s Apple Watch,2022-07-04 14:09:29 -0700,2022-07-04 13:36:16 -0700,2022-07-04 14:09:29 -0700,33.219770,NaN,NaN,NaN,NaN,NaN
4,eric,test@test.com,quantity,BodyMass,lb,iPhone (7),2023-05-07 13:35:43 -0700,2023-05-07 13:35:43 -0700,2023-05-07 13:35:43 -0700,150.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712020,eric,test@test.com,quantity,HeartRateVariabilitySDNN,ms,Eric’s Apple Watch,2024-03-17 02:09:02 -0700,2024-03-17 02:08:01 -0700,2024-03-17 02:09:00 -0700,39.133900,NaN,NaN,NaN,NaN,NaN
1712021,eric,test@test.com,quantity,HeartRateVariabilitySDNN,ms,Eric’s Apple Watch,2024-03-17 04:09:00 -0700,2024-03-17 04:07:59 -0700,2024-03-17 04:08:59 -0700,429.662000,NaN,NaN,NaN,NaN,NaN
1712022,eric,test@test.com,quantity,HeartRateVariabilitySDNN,ms,Eric’s Apple Watch,2024-03-17 06:09:05 -0700,2024-03-17 06:08:04 -0700,2024-03-17 06:09:04 -0700,54.801700,NaN,NaN,NaN,NaN,NaN
1712023,eric,test@test.com,quantity,HeartRateVariabilitySDNN,ms,Eric’s Apple Watch,2024-03-17 08:10:45 -0700,2024-03-17 08:09:44 -0700,2024-03-17 08:10:43 -0700,56.816700,NaN,NaN,NaN,NaN,NaN
